# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup as bs

In [19]:
soup = bs(html, 'lxml')
links = soup.findAll('link')
links = set(links)
links = list(links)
links

[<link href="//creativecommons.org/licenses/by-sa/3.0/" rel="license"/>,
 <link href="/w/index.php?title=Data_science&amp;action=edit" rel="alternate" title="Edit this page" type="application/x-wiki"/>,
 <link href="/w/load.php?lang=en&amp;modules=site.styles&amp;only=styles&amp;skin=vector" rel="stylesheet"/>,
 <link href="/static/apple-touch/wikipedia.png" rel="apple-touch-icon"/>,
 <link href="https://en.wikipedia.org/wiki/Data_science" rel="canonical"/>,
 <link href="/static/favicon/wikipedia.ico" rel="shortcut icon"/>,
 <link href="mw-data:TemplateStyles:r935243608" rel="mw-deduplicated-inline-style"/>,
 <link href="android-app://org.wikipedia/http/en.m.wikipedia.org/wiki/Data_science" rel="alternate"/>,
 <link href="/w/load.php?lang=en&amp;modules=ext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cmediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.skinning.interface%7Cmediawiki.toc.styles%7Cskins.vector.styles%7Cwikibas

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [16]:
domain = 'http://wikipedia.org'

In [20]:
absolute = [link['href'] for link in links if link['href'].startswith('http') and '%' not in link['href']]
print(absolute)
relative = [domain+link['href'] for link in links if not link['href'].startswith('http')]
print(relative)
links_clean = absolute + relative
links_clean

['https://en.wikipedia.org/wiki/Data_science']
['http://wikipedia.org//creativecommons.org/licenses/by-sa/3.0/', 'http://wikipedia.org/w/index.php?title=Data_science&action=edit', 'http://wikipedia.org/w/load.php?lang=en&modules=site.styles&only=styles&skin=vector', 'http://wikipedia.org/static/apple-touch/wikipedia.png', 'http://wikipedia.org/static/favicon/wikipedia.ico', 'http://wikipedia.orgmw-data:TemplateStyles:r935243608', 'http://wikipedia.organdroid-app://org.wikipedia/http/en.m.wikipedia.org/wiki/Data_science', 'http://wikipedia.org/w/load.php?lang=en&modules=ext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cmediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.skinning.interface%7Cmediawiki.toc.styles%7Cskins.vector.styles%7Cwikibase.client.init&only=styles&skin=vector', 'http://wikipedia.org/w/opensearch_desc.php', 'http://wikipedia.org//meta.wikimedia.org', 'http://wikipedia.org//en.wikipedia.org/w/api.php?action=r

['https://en.wikipedia.org/wiki/Data_science',
 'http://wikipedia.org//creativecommons.org/licenses/by-sa/3.0/',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit',
 'http://wikipedia.org/w/load.php?lang=en&modules=site.styles&only=styles&skin=vector',
 'http://wikipedia.org/static/apple-touch/wikipedia.png',
 'http://wikipedia.org/static/favicon/wikipedia.ico',
 'http://wikipedia.orgmw-data:TemplateStyles:r935243608',
 'http://wikipedia.organdroid-app://org.wikipedia/http/en.m.wikipedia.org/wiki/Data_science',
 'http://wikipedia.org/w/load.php?lang=en&modules=ext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cmediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.skinning.interface%7Cmediawiki.toc.styles%7Cskins.vector.styles%7Cwikibase.client.init&only=styles&skin=vector',
 'http://wikipedia.org/w/opensearch_desc.php',
 'http://wikipedia.org//meta.wikimedia.org',
 'http://wikipedia.org//en.wikipedia.org/w/api.ph

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [7]:
import os

In [ ]:
os.mkdir('wikipedia')

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [8]:
from slugify import slugify

In [65]:
def index_page(link):
    try:
        html = requests.get(link, headers={'User-Agent':'Mozilla/5.0 (Windows; U; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 2.0.50727)'})
        if html.status_code == 200:
            filename = slugify(link) + '.html'
            f = open('./wikipedia/'+filename, "a")
            f.write(str(html.content))
            f.close()
        else:
            print('Status code:', html.status_code)
    except:
        print('Link "{}" not working'.format(link))

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [70]:
%%time
for link in links_clean:
    index_page(link)

Status code: 404
Link "http://wikipedia.orgmw-data:TemplateStyles:r935243608" not working
Link "http://wikipedia.org/w/load.php?lang=en&modules=ext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cmediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.skinning.interface%7Cmediawiki.toc.styles%7Cskins.vector.styles%7Cwikibase.client.init&only=styles&skin=vector" not working
Status code: 404
Status code: 404
Status code: 404
CPU times: user 1.54 s, sys: 30.8 ms, total: 1.57 s
Wall time: 10.4 s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [67]:
import multiprocessing

In [69]:
%%time
pool = multiprocessing.Pool()
result = pool.map(index_page, links_clean)
pool.terminate()
pool.join()
print(result)

Link "http://wikipedia.orgmw-data:TemplateStyles:r935243608" not working
Status code: 404
Link "http://wikipedia.org/w/load.php?lang=en&modules=ext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cmediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.skinning.interface%7Cmediawiki.toc.styles%7Cskins.vector.styles%7Cwikibase.client.init&only=styles&skin=vector" not working
Status code: 404
Status code: 404
Status code: 404
[None, None, None, None, None, None, None, None, None, None, None, None, None, None]
CPU times: user 49.9 ms, sys: 15.5 ms, total: 65.4 ms
Wall time: 2.56 s


# :O